In [1]:
import os
import pandas as pd
import datetime as dt
import numpy as np
import re

In [2]:
today = pd.Timestamp(dt.datetime.today().date())
today_ymd = today.strftime('%Y%m%d')
today_y_m_d = today.strftime('%Y_%m_%d')

yesterday = today - dt.timedelta(days=1)
yesterday_ymd = yesterday.strftime('%Y%m%d')
yesterday_y_m_d = yesterday.strftime('%Y-%m-%d')

tomorrow = today + dt.timedelta(days=1)
tomorrow_ymd = tomorrow.strftime('%Y%m%d')
tomorrow_y_m_d = tomorrow.strftime('%Y-%m-%d')

print(today_y_m_d)

2022_09_08


In [3]:
column_names = {'Emiasid': 'emiasid', 'Emiasid ': 'emiasid' , 'emiasid ': 'emiasid', 'EMIAS_ID': 'emiasid', 'МО': 'mo', 
                'МУ': 'mu', 'Логин оператора': 'login', 'Дата и время звонка': 'call_date', 'Результат звонка': 'call_result',
                'Результат звонка ': 'call_result', 'Разговор состоялся': 'talk_result', 'Комменатрий': 'comment', 
                'Дата и время переноса': 'next_call_date', 'Контактный телефон': 'phone_number', 'ФИО врача': 'doctor_fio', 
                'ФИО помощника врача': 'assistant_fio', 'Логин помощника врача': 'login', 'Зал №': 'hall_number',
                'Логин naumen': 'login', 'МО закрепления': 'mo', 'Отсутствует на рабочем месте': 'absence'}

In [23]:
# путь папки с данными по звонкам
calls_data_path = r'\Users\murad_satabaev\Documents\all_tasks_in_one_place'
calls_data_path = calls_data_path.split('\\')
calls_data_path = os.path.join('C:\\', r'\\', *calls_data_path)

In [24]:
# путь файла с текущими статусами пациентов
patients_current_statuses_path = r'\\t999\Сетевой диск\pdn\data\patients_current_statuses_path'
patients_current_statuses_path = patients_current_statuses_path.split('\\')
patients_current_statuses_path = os.path.join(r'\\', *patients_current_statuses_path)
print(patients_current_statuses_path)

\\t999\Сетевой диск\pdn\data\patients_current_statuses_path


In [25]:
calls_data_path = os.path.join(calls_data_path, 'calls_data_20220902_today.xlsx')

In [44]:
calls_data_reading = pd.read_excel(calls_data_path)
calls_data_reading.shape[0]

62643

In [45]:
calls_data_reading = calls_data_reading.rename(columns=column_names)

In [47]:
calls_data_reading['emiasid'] = calls_data_reading['emiasid'].astype(str).str.replace('\.0', '')
calls_data_reading['call_date'] = calls_data_reading['call_date'].apply(lambda x: pd.to_datetime(x, dayfirst=True).date())
calls_data_reading['next_call_date'] = \
calls_data_reading['next_call_date'].apply(lambda x: pd.to_datetime(x, dayfirst=True, errors='coerce').date())

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_7136\406665173.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  calls_data_reading['emiasid'] = calls_data_reading['emiasid'].astype(str).str.replace('\.0', '')


In [48]:
monthes = {'august', 'september'}

In [49]:
for month in monthes:
    
    
    if 'september' in month:
        calls_data = calls_data_reading[calls_data_reading.call_date >= dt.date(2022, 9, 2)].reset_index(drop=True).copy()
    else:
        calls_data = calls_data_reading.copy()
        
    
    emiasid_touches = \
    calls_data.groupby('emiasid').agg({'call_date': 'count'}).rename(columns={'call_date': 'calls_amount'}).reset_index()
    
    last_calls_data = \
    calls_data.sort_values(by='call_date').drop_duplicates(subset=['emiasid'], keep='last').reset_index(drop=True)
    
    last_calls_data = last_calls_data[['emiasid', 'call_date', 'call_result', 'talk_result', 'next_call_date', 'comment']]
    
    last_calls_data = last_calls_data.rename(
        columns={'comment': 'old_comment', 'call_date': 'last_call_date', 'call_result': 'last_call_result',
                'talk_result': 'last_talk_result'})
    
    last_calls_data['emiasid'] = last_calls_data['emiasid'].astype(str).str.replace('\.0', '')
    last_calls_data['last_call_date'] = last_calls_data['last_call_date'].apply(lambda x: pd.to_datetime(x, dayfirst=True).date())
    last_calls_data['next_call_date'] = \
    last_calls_data['next_call_date'].apply(lambda x: pd.to_datetime(x, dayfirst=True).date())
    
    
    patients_current_statuses_reading = pd.read_excel(
        os.path.join(patients_current_statuses_path, rf'{month}_patients_current_statuses_{today_y_m_d}.xlsx'))
    
    
    patients_current_statuses = patients_current_statuses_reading.rename(columns=column_names).copy()
    
    patients_current_statuses['emiasid'] = patients_current_statuses['emiasid'].astype(str).str.replace('\.0', '')
    patients_current_statuses['date'] = \
    patients_current_statuses['date'].apply(lambda x: pd.to_datetime(x, dayfirst=True).date())
    
    patients_current_statuses_merged = patients_current_statuses.merge(emiasid_touches, on=['emiasid'], how='left')
    patients_current_statuses_merged = patients_current_statuses_merged.merge(last_calls_data, on=['emiasid'], how='left')
    patients_current_statuses_merged.to_excel(
        os.path.join('C:\\', 'Users', 'murad_satabaev', 'Documents', 'all_tasks_in_one_place',
        'patients_current_statuses_merged', f'{month}_patients_current_statuses_merged_{today_y_m_d}.xlsx'), index=False)

C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_7136\2959715948.py:22: FutureWarning: The default value of regex will change from True to False in a future version.
  last_calls_data['emiasid'] = last_calls_data['emiasid'].astype(str).str.replace('\.0', '')
C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_7136\2959715948.py:34: FutureWarning: The default value of regex will change from True to False in a future version.
  patients_current_statuses['emiasid'] = patients_current_statuses['emiasid'].astype(str).str.replace('\.0', '')
C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_7136\2959715948.py:22: FutureWarning: The default value of regex will change from True to False in a future version.
  last_calls_data['emiasid'] = last_calls_data['emiasid'].astype(str).str.replace('\.0', '')
C:\Users\murad_satabaev\AppData\Local\Temp\ipykernel_7136\2959715948.py:34: FutureWarning: The default value of regex will change from True to False in a future version.
  patients_current_st